In [1]:
import functools
import random
import textwrap

from huggingface_hub import InferenceApi

In [2]:
class CompletionError(Exception):
    """An error in attempting to complete text with BLOOM."""
    pass

In [3]:
class UnexpectedResponseError(AssertionError):
    """An unexpected error completing text. Probably a bug."""
    pass

In [4]:
class Supplier:
    """As the value of a BLOOM parameter, is called instead of used."""

    __slots__ = ('_func',)

    __match_args__ = ('_func',)

    def __init__(self, func):
        self._func = func
    
    def __repr__(self):
        return f'{type(self).__name__}({self._func!r})'
    
    def __call__(self):
        return self._func()

In [5]:
@functools.cache
def _get_inference_api():
    with open('.hf_token', encoding='utf-8') as file:
        api_token = file.read().strip()

    return InferenceApi(repo_id='bigscience/bloom', token=api_token)

In [13]:
class Completer:
    """High-level interface to Bloom completion."""

    __slots__ = ('_inference', '_text', '__dict__')

    def __init__(self, prompt, **parameters):
        """Create a completer for the given prompt and optional parameters."""
        self._inference = _get_inference_api()
        self.text = prompt

        if any(name.startswith('_') for name in parameters):
            raise TypeError('cannot set parameter name with leading "_"')
        
        self._set_defaults()
        self.__dict__.update(parameters)

    def __str__(self):
        """Prettified version of the text stored so far."""
        raw = self.text.strip().split('\n')
        pretty = ('\n'.join(textwrap.wrap(graf)) for graf in raw)
        return '\n\n'.join(pretty)
    
    @property
    def text(self):
        """The prompt plus any appended completions."""
        return self._text
    
    @text.setter
    def text(self, value):
        if not isinstance(value, str):
            raise TypeError('prompt must be a string')
        if not value:
            raise ValueError('prompt must be nonempty')
        self._text = value

    def __call__(self):
        """Attempt to complete the stored text, and print all text so far."""
        self.complete()
        print(self)
    
    def complete(self):
        """Attempt to complete the stored text, but do not print anything."""
        match self._inference(inputs=self.text, params=self._build_params()):
            case [{'generated_text': completion}]:
                self._text = completion  # Whatever BLOOM returns, we accept.
            case {'error': [*errors]}:
                raise CompletionError(*errors)
            case other_response:
                raise UnexpectedResponseError(other_response)
    
    def _set_defaults(self):
        self.do_sample = True
        self.max_new_tokens = 250
        self.seed = Supplier(random.random)
        self.temperature = 0.75
    
    def _build_params(self):
        return {name: (value() if isinstance(value, Supplier) else value)
                for name, value in sorted(self.__dict__.items())}


In [14]:
bloom = Completer('Sam opened the door and his jaw dropped. Here')

In [15]:
bloom()

Sam opened the door and his jaw dropped. Here he was sitting on a
large and imposing bed. A large desk sat to his right. Two large and
antique looking lamps sat at the bottom of the desk. A large, black,
leather bound book sat open on the desk. At the far end of the room
stood a large, antique looking wardrobe. It appeared to be made of
solid oak. One of the doors was open and it displayed the most
exquisite collection of clothes Sam had ever seen. The clothes hung on
a rack and there were several pairs of shoes. The clothes had a real
‘glamour’ about them. They were beautiful and exquisite. They were
also very expensive.

Sam looked round the room and was seriously impressed. It was like a
scene from one of those old movies. He wondered what he’d look like
dressed up in these clothes. He saw that one of the handbags on the
wardrobe was open and looked inside. It contained a small, silver
revolver. He looked out of the window. It was a small, metal window.
Looking through the window he